In [1]:
import os
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# 모델 로드
model = YOLO('/content/drive/MyDrive/DL_Project/model_backup (1)/runs/detect/train4/weights/best.pt')  # 학습된 모델 경로로 변경

# 비디오 파일 경로 및 출력 경로 설정
video_path = r'/content/drive/MyDrive/DL_Project/test_data.mp4'
output_video_path = r'/content/output_video.avi'
capture_dir = r'/content/captured_images/'

# 디렉토리 존재 확인 및 생성
os.makedirs(capture_dir, exist_ok=True)

# 비디오 파일 열기
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Unable to open video file at {video_path}")
    exit()

# 비디오 작성기 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# 캡처 플래그 및 카운터 초기화
capture_done = False
capture_counter = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임을 모델에 입력
    results = model(frame)

    # 결과 처리 및 비디오 프레임에 오버레이
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # 바운딩 박스 좌표
        confidences = result.boxes.conf.cpu().numpy()  # 신뢰도
        classes = result.boxes.cls.cpu().numpy()  # 클래스

        for box, conf, cls in zip(boxes, confidences, classes):
          if conf >= 0.8:
            x1, y1, x2, y2 = map(int, box)
            label = f'{model.names[int(cls)]} {conf:.2f}'

            # 객체를 프레임에 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # 조건을 만족하고 캡처가 아직 진행되지 않았을 때
            if not capture_done and conf >= 0.8:
                capture_counter += 1
                obj_image_path = os.path.join(capture_dir, f'captured_object_{capture_counter}.png')
                cv2.imwrite(obj_image_path, frame)

                # 캡처 완료 플래그 설정
                capture_done = True
                print(f"Captured image saved to {obj_image_path}")

    # 결과를 비디오 파일로 저장
    out.write(frame)

    # 화면에 표시 (디버깅용)
    cv2_imshow(frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video saved to {output_video_path}")


ModuleNotFoundError: No module named 'ultralytics'